# Tiny A11y Model Fine-tune Notebook
This notebook fine-tunes DeepSeek-Coder on WCAG + MDN accessibility datasets.

Requirements: Free GPU runtime in Colab.

In [23]:
# Install required packages
!pip install --upgrade pip
!pip install datasets transformers peft huggingface_hub accelerate

In [24]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [25]:
# Load dataset from Hugging Face
from datasets import load_dataset
dataset = load_dataset('younglim/a11y-dataset', split='train')
print(dataset[0])

{'text': "Contributions to this repository are intended to become part of Recommendation-track documents governed by the\n[W3C Patent Policy](https://www.w3.org/Consortium/Patent-Policy-20040205/) and\n[Document License](https://www.w3.org/Consortium/Legal/copyright-documents). To make substantive contributions to specifications, you must either participate\nin the relevant W3C Working Group or make a non-member patent licensing commitment.\n\nIf you are not the sole contributor to a contribution (pull request), please identify all \ncontributors in the pull request comment.\n\nTo add a contributor (other than yourself, that's automatic), mark them one per line as follows:\n\n```\n+@github_username\n```\n\nIf you added a contributor by mistake, you can remove them in a comment with:\n\n```\n-@github_username\n```\n\nIf you are making a pull request on behalf of someone else but you had no part in designing the \nfeature, you can remove yourself with the above syntax.\n\nPublic comments

In [26]:
# Load model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = 'deepseek-ai/deepseek-coder-350m-instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# LoRA config for lightweight fine-tuning
peft_config = LoraConfig(r=8, lora_alpha=16, target_modules=['q_proj','v_proj'])
model = get_peft_model(model, peft_config)

In [27]:
# Tokenize dataset and add labels for causal LM
def tokenize(example):
    tok = tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)
    tok['labels'] = tok['input_ids'].copy()
    return tok

dataset = dataset.map(tokenize, batched=True)

In [28]:
# Setup Trainer and TrainingArguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    push_to_hub=True,
    hub_model_id='younglim/tiny-a11y-model'
)

trainer = Trainer(model=model, args=training_args, train_dataset=dataset)

OutOfMemoryError: CUDA out of memory. Tried to allocate 44.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 20.12 MiB is free. Process 2662 has 14.72 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 60.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Push the fine-tuned model to Hugging Face Hub
trainer.push_to_hub()

🎉 After running all cells, your fine-tuned Tiny A11y Model will be available at:
[https://huggingface.co/younglim/tiny-a11y-model](https://huggingface.co/younglim/tiny-a11y-model)